Digital elevation model (DEM) workflows
=================================

.. admonition:: Overview

   In this tutorial we demonstrate how to incorporate DEM data into Eradiate simulations.

.. admonition:: Prerequisites

   * Basic knowledge of the Eradiate workflow (see :doc:`../getting_started/eradiate_quickstart`).
   * Eradiate expert interface

.. admonition:: What you will learn

   * What format Eradiate expects for DEM data
   * How to instantiate a DEMSurface
   * How to compute the BRF of a DEM based surface with and without atmosphere
   * How to compute a bottom-of-atmosphere (BOA) RGB image on top of the DEM surface.

----

## Input data format

Eradiate represents DEM data as triangulated meshes. The easiest and preferred way to provide DEM data to Eradiate is therefore a prepared triangulated mesh in PLY-format.

Eradiate does not parse metadata associated with the PLY-format and therefore will not perform any unit conversions on the data. Vertex position data will be interpreted as kernel-units and the meaning of the x and y dimensions will depend on the scene geometry.

For example the default case, the plane parallel geometry and default kernel units, mean that positions will be interpreted as meters and the X, Y, and Z dimensions will map to EAST, NORTH, and UP (ENU).

The rest of this tutorial will assume that DEM data in ply format is available. In this tutorial we will use a 20 km by 20 km section near the Libya4 CEOS site. (Note for reviewer: Is this the right name?) If this file is not present on your machine, it can be downloaded from (add a link) and must be placed in the (add) subfolder.

## Imports

First we import all the necessary components for this tutorial

In [ ]:
import eradiate

eradiate.set_mode("mono")

from eradiate.units import unit_registry as ureg

import matplotlib.pyplot as plt
import numpy as np

## Declaring scene elements

Now we declare the scene elements that are independent of the DEM. That is, illumination, sensor and atmosphere.
In this first part of the tutorial we will compute the BRF of the scene first without and then with an atmosphere. We will limit the observation to the principal plane.
We will attach a Lamberitian BSDF to all surfaces in this tutorial, to visualize the effect of the 3D terrain on the BRF.

In [ ]:
illumination = eradiate.scenes.illumination.DirectionalIllumination(zenith=60, azimuth=0)

camera_iso = eradiate.scenes.measure.PerspectiveCameraMeasure(
    origin=[300, 300, 300]*ureg.km,
    target=[0, 0, 0]*ureg.km,
    up=[0, 0, 1],
    film_resolution=[128, 128],
    srf={"type": "delta", "wavelengths": [550.0] * ureg.nm},
    fov=150,
)

flat_surface = eradiate.scenes.bsdfs.LambertianBSDF(reflectance={
    "type": "interpolated",
    "wavelengths": [440, 500, 660],
    "values": [0.3, 0.5, 0.7]
})

Now we visualize the basic scene we have created

In [ ]:
exp = eradiate.experiments.AtmosphereExperiment(
    illumination=illumination,
    measures=camera_iso,
    surface=flat_surface,
    atmosphere=None
)

result = eradiate.run(exp, spp=16)
plt.imshow(result.radiance.squeeze(), cmap="gray")

A flat surface with Lamberitan reflectance and no atmosphere. Not very interesting, but as expected. 

### Adding the DEM
Now we add the DEM mesh. We achive this by using Eradiate's expert interface.

To make the code easier to read, we define a separate scene parameter function, that will return the material's reflectance, based on the simulated wavelength. The function supports three wavelengths, which will be used later in this tutorial, to create an RGB image.

In [ ]:
from eradiate import KernelContext
from eradiate.kernel import UpdateParameter, scene_parameter

In [ ]:
@scene_parameter(flags=UpdateParameter.Flags.SPECTRAL)
def lambertian_sand(ctx: KernelContext):
    if ctx.si.w == 440.0 * ureg.nm:
        return 0.3
    elif ctx.si.w == 550.0 * ureg.nm:
        return 0.5
    elif ctx.si.w == 660.0 * ureg.nm:
        return 0.7
    else:
        raise RuntimeError(f"Unsupported context: {ctx}")
        return np.expand_dims(np.ones_like(lc_map)*0.5, -1)

In [ ]:
dem_dict={
    "diffuse_material": {
        "type": "diffuse", 
        "id": "dem_mat",
        "reflectance": {
            "type": "uniform",
            "value": 0.0
        }
    },
    "mesh": {
        "type": "ply", 
        "id": "surface_dem_mesh", 
        "bsdf": {"type": "ref", "id": "dem_mat"},
        "filename": "/home/schunkes/src/eradiate-data/tutorials/dem_workflow/libya4_20km.ply"  # make this a path relative to the notebook
    },
}
dem_pmap={
    "dem_mat.reflectance.value": lambertian_sand
}

Additionally, we define a new perspective camera, that looks at the scene from the zenith. This way, the shadows cast by the 3D surface will be easily visible.

In [ ]:
camera_top = eradiate.scenes.measure.PerspectiveCameraMeasure(
    origin=[0, 0, 30]*ureg.km,
    target=[0, 0, 0]*ureg.km,
    up=[0, 1, 0],
    film_resolution=[256, 256],
    srf={"type": "delta", "wavelengths": [550.0] * ureg.nm},
    fov=50,
)

Now we add the scene dictionary and the parameter map to the experiment and render again at 550nm.

In [ ]:
exp = eradiate.experiments.AtmosphereExperiment(
    illumination=illumination,
    measures=camera_top,
    surface=flat_surface,
    atmosphere=None,
    kdict=dem_dict,
    kpmap=dem_pmap
)

result = eradiate.run(exp, spp=16)
plt.axis("off")
plt.imshow(result.radiance.squeeze(), cmap="gray")
plt.show()

We can clearly see the dunes of the Libya4 site. 

### Computing the BRF

Now we compute the BRF of this patch, first without then with an atmosphere present. Will will focus on the principal plane. First we define a suitable measure and add it to the experiment, instead of the perspective camera.

In [ ]:
brf_sensor = eradiate.scenes.measure.MultiDistantMeasure.hplane(
    azimuth=0,
    zeniths=np.arange(-80, 81, 2),
    srf={"type": "delta", "wavelengths": [550.0] * ureg.nm},
    target=eradiate.scenes.measure.TargetRectangle(
        xmin=-10*ureg.km,
        xmax=10*ureg.km,
        ymin=-10*ureg.km,
        ymax=10*ureg.km,
        z=130*ureg.m
    ),
)

In [ ]:
exp = eradiate.experiments.AtmosphereExperiment(
    illumination=illumination,
    measures=brf_sensor,
    surface=flat_surface,
    atmosphere=None,
    kdict=dem_dict,
    kpmap=dem_pmap
)

result = eradiate.run(exp, spp=4**8)
plt.plot(result.vza.squeeze(), result.brf.squeeze(), label="DEM Surface")
plt.hlines(xmin=-80, xmax=80, y=0.5, ls="--", color="gray", label="Lambertian reference")
plt.ylim(0.47, 0.53)
plt.xlabel("VZA [deg]")
plt.ylabel("BRF [-]")
plt.legend()
plt.show()

The reflectance of the DEM surface has clear directional features. In the backward scattering direction, reflectance is increased, while in the forward scattering direction, it is decreased. As a comparison, the dashed line denotes the expected reflectance of the Lambertian surface without 3D terrain.

### With an atmosphere

Now, to complete the scene, we add an atmosphere. We define an atmosphere with two components, the base molecular atmosphere and a simple aerosol layer, that extends from 0 m to 2 km elevation.

In [ ]:
atmosphere = eradiate.scenes.atmosphere.HeterogeneousAtmosphere(
    molecular_atmosphere=eradiate.scenes.atmosphere.MolecularAtmosphere()
)

In [ ]:
exp_dem = eradiate.experiments.AtmosphereExperiment(
    illumination=illumination,
    measures=sensor,
    surface=flat_surface,
    atmosphere=atmosphere,
    kdict=dem_dict,
    kpmap=dem_pmap
)

result_dem = eradiate.run(exp_dem, spp=4**10)
plt.plot(result_dem.vza.squeeze(), result_dem.brf.squeeze(), label="DEM Surface")
plt.xlabel("VZA [deg]")
plt.ylabel("BRF [-]")
plt.legend()
plt.show()

The effect of the atmosphere is much larger than that of the DEM surface and we can not as easily compare this simulation to another case. Therefore, we run another simulation, where no DEM is present and compare those two results. We need to adjust the flat surface for this simulation. The mean elevation of the Libya4 site is circa 130m, so we need to define a new surface object with its elevation adjusted accordingly.

In [ ]:
flat_surface_elevated = eradiate.scenes.surface.BasicSurface(
    shape=eradiate.scenes.shapes.RectangleShape(
        center=[0, 0, 130]*ureg.m  # specify ground elevation using Geometry
    ),
    bsdf=eradiate.scenes.bsdfs.LambertianBSDF(reflectance={
        "type": "interpolated",
        "wavelengths": [440, 500, 660],
        "values": [0.3, 0.5, 0.7]
    })
)

In [ ]:
exp_flat = eradiate.experiments.AtmosphereExperiment(
    illumination=illumination,
    measures=sensor,
    surface=flat_surface_elevated,
    atmosphere=atmosphere,
    kdict={},
    kpmap={}
)

result_flat = eradiate.run(exp_flat, spp=4**10)

fig, ax = plt.subplots(1, 2, figsize=(6,3), layout="constrained")
ax[0].plot(result_dem.vza.squeeze(), result_dem.brf.squeeze(), label="DEM Surface")
ax[0].plot(result_flat.vza.squeeze(), result_flat.brf.squeeze(), label="Flat Surface", ls="--")
ax[0].set_xlabel("VZA [deg]")
ax[0].set_ylabel("BRF [-]")
ax[0].legend()

ax[1].plot(result_dem.vza.squeeze(), result_dem.brf.squeeze()-result_flat.brf.squeeze())
ax[1].set_xlabel("VZA [deg]")
ax[1].set_ylabel("Absolute difference [-]")
plt.show()

To compare these two results, we plot the absolute difference between them. Both the increased reflectance in the backward scattering and the reduced reflectance in the forward scattering directions are visible in the difference between the two results.

### BOA RGB

To round this tutorial off, we will compute an RGB image, with the camera at the bottom of the atmosphere, looking across the dunes. To do this, we first define a new perspective camera with the desired position and orientation.

In [ ]:
from eradiate.xarray.interp import dataarray_to_rgb

In [ ]:
camera_oblique = eradiate.scenes.measure.PerspectiveCameraMeasure(
    origin=[-10, -10, 0.5]*ureg.km,
    target=[0, 0, 0]*ureg.km,
    up=[0, 0, 1],
    film_resolution=[320, 240],
    srf={"type": "multi_delta", "wavelengths": [440.0, 550.0, 660.0] * ureg.nm},
    fov=70,
)

Now we create an Eradiate experiment again, run the simulation and use a helper function to create an RGB image.

In [ ]:
exp = eradiate.experiments.AtmosphereExperiment(
    illumination=illumination,
    measures=camera_oblique,
    surface=flat_surface,
    atmosphere=atmosphere,
    kdict=dem_dict,
    kpmap=dem_pmap
)

result = eradiate.run(exp, spp=512)

img = dataarray_to_rgb(
    result.radiance, 
    channels=[("w", 660), ("w", 550), ("w", 440)]
)
plt.axis("off")
plt.imshow(img)
plt.show()

----

Final words
-----------

In this tutorial, we have learned, how to assemble a scene with a 3D surface using DEM data. We have computed the BRF of this scene with and without an atmosphere and compared the results to those of a flat surface.
Finally, we have created an RGB image at the bottom of atmosphere, illustrating atmospheric scattering and the 3D structure of the DEM surface.

Further reading
---------------

The documentation of :mod:`eradiate.scenes.surface` contains a function, that can convert elevation data to triangulated meshes, suitable for usage in Eradiate.